# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lerwick,60.1545,-1.1494,48.20,81,100,29.84,GB,1706401794
1,1,hermanus,-34.4187,19.2345,73.80,73,100,2.91,ZA,1706401795
2,2,stanley,54.8680,-1.6985,41.29,86,0,3.44,GB,1706401795
3,3,penalva,-3.2942,-45.1736,77.07,87,100,3.74,BR,1706401795
4,4,kangding,30.0513,101.9603,24.66,85,44,4.54,CN,1706401795


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriNatGeo",
    frame_width=700,
    frame_height=500,
    size="Humidity",
    scale=0.90,
    color="City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [30]:
# Narrow down cities that fit criteria and drop any results with null values
# Drop any rows with null values
# Display sample data
ideal_df = city_data_df[(city_data_df['Max Temp']>25)&
             (city_data_df['Max Temp']<79)&
             (city_data_df.Cloudiness==0)&
             (city_data_df.Humidity<50)]
ideal_df





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,ushuaia,-54.8000,-68.3000,53.26,47,0,11.50,AR,1706401800
37,37,calama,-22.4667,-68.9333,70.29,32,0,20.71,CL,1706401805
110,110,zouerate,22.7187,-12.4521,63.23,20,0,7.67,MR,1706401826
134,134,los llanos de aridane,28.6585,-17.9182,63.12,30,0,8.99,ES,1706401833
136,136,casper,42.8666,-106.3131,39.79,48,0,17.27,US,1706401833
158,158,ghardaia,32.4909,3.6735,61.95,25,0,0.00,DZ,1706401837
230,230,corumba,-19.0092,-57.6533,77.00,30,0,2.95,BR,1706401858
248,248,bouza,14.4229,6.0428,60.04,29,0,12.71,NE,1706401862
254,254,illizi,26.4833,8.4667,53.40,30,0,3.09,DZ,1706401863
351,351,robore,-18.3333,-59.7500,76.84,33,0,4.36,BO,1706401891


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# Display sample data

hotel_df = ideal_df.copy()
hotel_df = hotel_df[['City','Country','Lat','Lng']].copy()
hotel_df['Hotel Name'] =''
hotel_df

,City,Country,Lat,Lng,Hotel Name
28,ushuaia,AR,-54.8000,-68.3000,
37,calama,CL,-22.4667,-68.9333,
110,zouerate,MR,22.7187,-12.4521,
134,los llanos de aridane,ES,28.6585,-17.9182,
136,casper,US,42.8666,-106.3131,
158,ghardaia,DZ,32.4909,3.6735,
230,corumba,BR,-19.0092,-57.6533,
248,bouza,NE,14.4229,6.0428,
254,illizi,DZ,26.4833,8.4667,
351,robore,BO,-18.3333,-59.7500,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'radius': radius,
    'apiKey': geoapify_key

}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude, longitude = row['Lat'], row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle:{longitude},{latitude},{radius}'
    params["bias"] = f'proximity:{longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
calama - nearest hotel: Apart Hotel Geotel Calama
zouerate - nearest hotel: فندق تيرس زمور
los llanos de aridane - nearest hotel: Valle Aridane
casper - nearest hotel: Days Inn by Wyndham Casper
ghardaia - nearest hotel: فندق بلعجال
corumba - nearest hotel: Hotel El Shadday
bouza - nearest hotel: No hotel found
illizi - nearest hotel: بوناقة
robore - nearest hotel: Lajas
illela - nearest hotel: HOTEL KADO
crane - nearest hotel: No hotel found
tan-tan - nearest hotel: Sable d´or
alaghsas - nearest hotel: Hôtel Zagado
puerto deseado - nearest hotel: Los Olmos


,City,Country,Lat,Lng,Hotel Name
28,ushuaia,AR,-54.8000,-68.3000,Apart Hotel Aires del Beagle
37,calama,CL,-22.4667,-68.9333,Apart Hotel Geotel Calama
110,zouerate,MR,22.7187,-12.4521,فندق تيرس زمور
134,los llanos de aridane,ES,28.6585,-17.9182,Valle Aridane
136,casper,US,42.8666,-106.3131,Days Inn by Wyndham Casper
158,ghardaia,DZ,32.4909,3.6735,فندق بلعجال
230,corumba,BR,-19.0092,-57.6533,Hotel El Shadday
248,bouza,NE,14.4229,6.0428,No hotel found
254,illizi,DZ,26.4833,8.4667,بوناقة
351,robore,BO,-18.3333,-59.7500,Lajas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display
# Configure the map plot
# Display the map

map_plot1 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    tiles="EsriNatGeo",
    frame_width=700,
    frame_height=500,
    size=500,
    scale=0.50,
    color="City",
    hover_cols=[ "Hotel Name", "Country"]
)

# Display the map
map_plot1



:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)